In [ ]:
import geopandas
import contextily as ctx

germany = geopandas.read_file('data/regions/germany.geo.json')
portugal = geopandas.read_file('data/regions/portugal_envelope.geo.json')
saxonia = geopandas.read_file('data/regions/roughly_saxonia.geo.json')

In [ ]:
ctx.add_basemap(germany.to_crs(epsg=3857).plot(figsize=(10, 10), alpha=0.3, edgecolor='k'))

In [ ]:
ctx.add_basemap(portugal.to_crs(epsg=3857).plot(figsize=(10, 10), alpha=0.3, edgecolor='k'))

In [ ]:
ctx.add_basemap(saxonia.to_crs(epsg=3857).plot(figsize=(10, 10), alpha=0.3, edgecolor='k'))

In [ ]:
f"Area of Germany approx: {germany.to_crs(epsg=8857).area[0] / 10**6:.1f} km²" # Use Equal Earth projection to calculate area size

In [ ]:
# Collection of Polygons (rectangles) to be put in GeoPandas data frame
collection = {
    "type": "FeatureCollection",
    "features": []
}
bin_width = 0.125 # degrees (defined by TEMIS)

# Fill the collection
with open("data/temis/no2_201808_clipped.asc") as data:
    lat = 0
    offset = -72 # Stay with me here, the data is structured that way, we need to go from -180° to +180°, offset is moving from -72 (-180/2.5) to 72 (180/2.5) for each latitude
    for line in data:
        if line.startswith("lat="):
            lat = float(line.split("=")[1]) - bin_width / 2
            offset = -72
        elif line.startswith('-') or line[0:4].strip().isdigit():
            count = 0 # There are twenty numbers per line, we need to keep track where we are
            for long in [x * 0.125 + (offset * 2.5) for x in range(0, 20)]:
                emission = int(line[count*4:count*4+4]) # All emission values are four digits wide
                if emission >= 0:
                    collection["features"].append({
                        "type": "Feature",
                        "properties": {"emission [1e13 molecules/cm²]": emission},
                        "geometry": {"type": "Polygon", "coordinates": [[(long, lat), (long + bin_width, lat), (long + bin_width, lat + bin_width), (long, lat + bin_width), (long, lat)]]},
                    })
                count += 1
            offset += 1

# Create dataframe and clip to Germany
no2 = geopandas.GeoDataFrame.from_features(collection, crs="EPSG:4326")

In [ ]:
ger_no2 = no2.cx[germany.total_bounds[0]:germany.total_bounds[2], germany.total_bounds[1]:germany.total_bounds[3]] # This is probably not really needed
ger_no2 = geopandas.overlay(ger_no2, germany, how='intersection')

# Draw this! And compare to https://www.temis.nl/airpollution/no2col/no2month_tropomi.php?Region=1&Year=2018&Month=08
ctx.add_basemap(ger_no2.to_crs(epsg=3857).plot("emission [1e13 molecules/cm²]", legend=True, figsize=(20, 20), alpha=0.6, edgecolor='k'))

In [ ]:
ger_no2["area [m²]"] = ger_no2.to_crs(epsg=5243).area
ger_no2["area [km²]"] = ger_no2["area [m²]"] / 10**6
ger_no2["area [cm²]"] = ger_no2["area [m²]"] * 10**4
ger_no2["emission [molecules]"] = ger_no2["emission [1e13 molecules/cm²]"] * ger_no2["area [cm²]"] * 10**13
ger_no2["emission [kg]"] = (ger_no2["emission [molecules]"] / (6.022 * 10**23)) * 46.01 / 1000
ger_no2

In [ ]:
dailyNo2 = ger_no2["emission [kg]"].sum() / 10**3
f"This computes to {dailyNo2:.1f}t of NO2 per day or {dailyNo2*31/10**3:.1f}kt for the month of August 2018."